In [2]:
import pandas as pd
import numpy as np
import os
import sys
import json
import requests
import gc
from tqdm import tqdm

from google.cloud import bigquery
from google.cloud.bigquery import job
from datetime import date, timedelta

PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)

import os


#  클라이언트 설정



# 실행

In [17]:
byshop_queries = []
shopby_queries = []

query_list = {
    'byshop' : byshop_queries,
    'shopby' : shopby_queries
}


for i, fd in enumerate(folders):
    files = os.listdir(f'sql/{folders[i]}')
    files = [f for f in files if f.endswith('.sql')]
    
    for file in files:
        with open(f'sql/{folders[i]}/{file}', 'r') as f:
            query =f.read()
            query_list[fd].append(query)



In [18]:
from datetime import datetime, date, timedelta
from google.cloud import bigquery
import time

def main_shopby(queries,historical_start_date,search_start_date):
    bq = bigquery.Client()
    
    
    # A: 데이터 셋의 시작 날짜 (고정)
    DATA_START_DATE = historical_start_date
    
    # B: 조회하려는 시작 날짜
    QUERY_START_DATE = search_start_date
    
    # C: 조회하려는 마감 날짜 (어제)
    QUERY_END_DATE = date.today() - timedelta(days=1)
    
    # 유효성 검사
    if QUERY_START_DATE < DATA_START_DATE:
        raise ValueError(f"조회 시작일({QUERY_START_DATE})은 데이터 시작일({DATA_START_DATE}) 이후여야 합니다.")
    if QUERY_END_DATE < QUERY_START_DATE:
        raise ValueError(f"조회 종료일({QUERY_END_DATE})은 조회 시작일({QUERY_START_DATE}) 이후여야 합니다.")
    
    # 날짜를 순차적으로 실행
    current_date = QUERY_START_DATE
    while current_date <= QUERY_END_DATE:
        cur_date = current_date.strftime('%Y-%m-%d')
        cur_date_plus = (current_date + timedelta(days=1)).strftime('%Y-%m-%d')
        
        print(f"\n=== {DATA_START_DATE.strftime('%Y-%m-%d')} 부터 {cur_date} 까지의 누적 데이터 처리 ===")
        print(f"기준일: {cur_date}, 다음날: {cur_date_plus}")

        # 각 쿼리를 개별적으로 실행
        for i, query in enumerate(queries, start=1):
            print(f"쿼리 {i} 실행 중...")
            try:
                formatted_query = query.format(
                    end_date=f"'{cur_date}'",
                    end_date_plus=f"'{cur_date_plus}'",
                    event_date_plus=f"'{cur_date}'"
                )
                
                job = bq.query(formatted_query)
                result = job.result()
                
                print(f"[쿼리 {i}] 성공")
                
            except Exception as e:
                print(f"[쿼리 {i}] 오류:", str(e))
                continue

        current_date += timedelta(days=1)

# query_list = {
#     'byshop' : byshop_queries,
#     'shopby' : shopby_queries
# }
tgt_query = "byshop"
search_start_date = date(2025, 1, 30) # 데이터 조회 날짜 기준.

if __name__ == "__main__":
    if tgt_query == "byshop":
        historical_start_date = date(2024, 3, 1)
    elif tgt_query == "shopby":
        historical_start_date = date(2024, 10, 1)
    else:
        historical_start_date = None
    total_start_time = time.time()
    
    main_shopby(query_list[tgt_query],historical_start_date,search_start_date)

    total_end_time = time.time()
    total_duration = total_end_time - total_start_time
    print(f"\n=== 전체 작업 완료 (총 소요시간: {total_duration:.2f}초) ===")



=== 2024-03-01 부터 2025-01-30 까지의 누적 데이터 처리 ===
기준일: 2025-01-30, 다음날: 2025-01-31
쿼리 1 실행 중...
[쿼리 1] 성공
쿼리 2 실행 중...
[쿼리 2] 성공
쿼리 3 실행 중...
[쿼리 3] 성공
쿼리 4 실행 중...
[쿼리 4] 성공
쿼리 5 실행 중...
[쿼리 5] 성공
쿼리 6 실행 중...
[쿼리 6] 성공
쿼리 7 실행 중...
[쿼리 7] 성공

=== 전체 작업 완료 (총 소요시간: 42.13초) ===
